In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
train=pd.read_csv('/kaggle/input/datathon2023/train.csv')
test=pd.read_csv('/kaggle/input/datathon2023/test_x.csv')
train

In [ ]:
test

In [ ]:
train.dtypes


In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
train.drop("index", axis=1, inplace=True)

In [ ]:
grouped = train.groupby(['Cinsiyet', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size, barmode='group')
fig.show()

In [ ]:
grouped = train.groupby(['Yaş Grubu', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size, barmode='group')
fig.show()

In [ ]:
grouped = train.groupby(['Medeni Durum', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size, barmode='group')
fig.show()

In [ ]:
grouped = train.groupby(['Eğitim Düzeyi', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size, barmode='group')
fig.show()

In [ ]:
grouped = train.groupby(['İstihdam Durumu', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size, barmode='group')
fig.show()

In [ ]:
grouped = train.groupby(['Yaşadığı Şehir', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size, barmode='group')
fig.show()

In [ ]:
grouped = train.groupby(['En Çok İlgilendiği Ürün Grubu', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size)
fig.show()

In [ ]:
grouped = train.groupby(['Eğitime Devam Etme Durumu', 'Öbek İsmi'])
grouped_size = grouped.size().unstack()

fig = px.bar(grouped_size)
fig.show()

In [ ]:
fig = px.box(train, x="Öbek İsmi", y="Yıllık Ortalama Gelir", color="Öbek İsmi")
fig.show()

In [ ]:
import plotly.express as px

grouped = train.groupby(['Öbek İsmi'])
grouped_mean = grouped['Yıllık Ortalama Gelir'].mean().reset_index()

fig = px.bar(grouped_mean, x='Öbek İsmi', y='Yıllık Ortalama Gelir')
fig.show()

In [ ]:
import plotly.express as px

grouped = train.groupby(['Öbek İsmi'])
grouped_mean = grouped['Yıllık Ortalama Satın Alım Miktarı'].mean().reset_index()

fig = px.bar(grouped_mean, x='Öbek İsmi', y='Yıllık Ortalama Satın Alım Miktarı')
fig.show()

In [ ]:
import plotly.express as px

grouped = train.groupby(['Öbek İsmi'])
grouped_mean = grouped['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].mean().reset_index()

fig = px.bar(grouped_mean, x='Öbek İsmi', y='Yıllık Ortalama Sipariş Verilen Ürün Adedi')
fig.show()

In [ ]:
import plotly.express as px

grouped = train.groupby(['Öbek İsmi'])
grouped_mean = grouped['Yıllık Ortalama Sepete Atılan Ürün Adedi'].mean().reset_index()

fig = px.bar(grouped_mean, x='Öbek İsmi', y='Yıllık Ortalama Sepete Atılan Ürün Adedi')
fig.show()

In [ ]:

# Preprocessing - Label Encoding
label_encoder = LabelEncoder()
object_columns = ['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi', 'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu', 'Eğitime Devam Etme Durumu']
for column in object_columns:
    train[column] = label_encoder.fit_transform(train[column])
    test[column] = label_encoder.transform(test[column])


In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(train.corr(), annot=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X_train = train.drop(['Öbek İsmi'], axis=1)
y_train = train['Öbek İsmi']

# Reorder the columns of the test data to match the order in X_train
X_test = test[X_train.columns]

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3]
}

# Create the RandomForestClassifier model
rf_model = RandomForestClassifier()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(rf_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
rf_model = RandomForestClassifier(**best_params)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)

accuracy = accuracy_score(y_train, rf_model.predict(X_train))
print("Accuracy Score:", accuracy)

submission = pd.DataFrame({'id': test['index'], 'Öbek İsmi': predictions})

submission.to_csv('submission.csv', index=False)